In [1]:
%load_ext autoreload
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

True

# Plain chroma tests

In [2]:
import chromadb
from chromadb.config import Settings

client = chromadb.PersistentClient(path="chromadb", 
                                   settings=Settings(allow_reset=True))
client.heartbeat()

1713515818559523004

In [3]:
collection = client.get_or_create_collection(name="finbooks")
collection.count()
# id = '01c62ecd-39ef-4da7-8d50-6a136de23705'

8440

In [9]:
# client.delete_collection("finbooks")

In [4]:
batch = collection.get(
    include=["metadatas"],
    limit=300,
    offset=100)

In [5]:
# client.delete_collection("finbooks")
%autoreload 2
#from pymongo import MongoClient
from raptor.raptor.chroma_manager import DatabaseConnectionManager
from raptor.raptor.tree_structures import Node

# Create a config with your custom models
db_path = "chromadb"
db_collection_name = 'trade_books'
mongodb_manager = DatabaseConnectionManager(db_config=(db_path, db_collection_name))

#mynode 

leaf_nodes = {}
for i in range(len(batch['ids'])):
    node_index = batch['metadatas'][i]['index']
    new_node = Node(text="sth",
                    index=node_index,
                    children=[None],
                    embeddings=[1,2,3],
                    db_manager=mongodb_manager,
                    node_id=batch['ids'][i])
    leaf_nodes[node_index] = new_node

print(len(leaf_nodes.values()))

2024-04-04 01:28:10,824 - Loading faiss with AVX2 support.
2024-04-04 01:28:10,852 - Successfully loaded faiss with AVX2 support.


300


In [6]:
from raptor.raptor.utils import get_node_list
from raptor.raptor.cluster_utils import RAPTOR_Clustering as rac
node_list = get_node_list(leaf_nodes)
cluster_embedding_model = "OpenAI"
nlist = rac.perform_clustering(
                node_list,
                mongodb_manager,
                cluster_embedding_model,
                reduction_dimension=10
            )

In [7]:
#collection = client.create_collection(name="test_collection")
collection.upsert(
    documents=["""This is a news article that talks about he government has extended the ban on the export of onions until further orders,
                according to the Ministry of Commerce and Industry. Earlier, the export of onions was prohibited till March 31, 2024. 
               The Directorate General of Foreign Trade (DGFT), a branch of the ministry responsible for handling export and import affairs, 
               has announced the extension in a notification dated March 22.""", "This is a book excerpt"],
    metadatas=[{"source": "directors"}, {"source": "producers"}],
    ids=["id1", "id2"]
)

In [11]:
results = collection.query(
    query_texts=["What about onions"],
    n_results=1
)

In [ ]:
import ingest
raptor_chroma = ingest.get_raptor_db_client()

In [3]:

text_path = "reference/sample/relval contents.txt"
ingest.ingest_file(text_path, "reference/sample", raptor_chroma)

2024-04-02 13:12:56,801 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptor.raptor.SummarizationModels.GPT3TurboSummarizationModel object at 0x7ff7c917f040>
            Embedding Models: {'OpenAI': <raptor.raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x7ff7c917f2e0>}
            Cluster Embedding Model: OpenAI
        
        Reduction Dimension: 10
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2024-04-02 13:12:56,804 - Successfully initialized ClusterTreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K:

Mongodb connection test: 1712041980739971803


True

In [7]:
collection.add(
    documents=["""After his arrest, Touseef Ali Farooqui was produced before the CJM’s court and remanded to 10-day custody of the STF. On the other hand, the STF apprehended another student of IIT, Guwahati, Sohail-ur Rahman, hailing from Jammu, for alleged links with ISIS.

On Saturday night, Farooqui was held by people in Damdama in Hajo and handed over to the police. The police then handed over Farooqui, a resident of Delhi, to the STF. He is said by sources to be a brilliant student.

The student is so religious-minded that he was seen reading the Quran in police custody as well."""],
    metadatas=[{"source": "news"}],
    ids=["id3"]
)

In [40]:
results = collection.query(
    query_texts=["What about news"],
    n_results=2
)
results
#client.reset()

{'ids': [['id2', 'id3']],
 'distances': [[1.7875811855962707, 1.791623107016882]],
 'metadatas': [[{'source': 'book'}, {'source': 'news'}]],
 'embeddings': None,
 'documents': [['This is a book excerpt',
   'After his arrest, Touseef Ali Farooqui was produced before the CJM’s court and remanded to 10-day custody of the STF. On the other hand, the STF apprehended another student of IIT, Guwahati, Sohail-ur Rahman, hailing from Jammu, for alleged links with ISIS.\n\nOn Saturday night, Farooqui was held by people in Damdama in Hajo and handed over to the police. The police then handed over Farooqui, a resident of Delhi, to the STF. He is said by sources to be a brilliant student.\n\nThe student is so religious-minded that he was seen reading the Quran in police custody as well.']],
 'uris': None,
 'data': None}

In [46]:
top_k = 2
results['distances'][0][:top_k]

[1.7875811855962707, 1.791623107016882]

# Apply chroma DB to raptor

In [3]:
with open('references/texts/treasury-bond-basis.txt', 'r') as file:
    text = file.read()

In [5]:
text[6999:10000]

"Trading When the CTD Is in Short Supply \n140 \nTrading the Calendar Spread \n142 \nFair Values for Treasury Note Calendar Spreads 143 \nProfiting from Mispricings in Calendar Spreads \n143 \nPatterns in Calendar Spreads 146 \nPractical Considerations in Trading the Basis \n147 \nix \n\x0cX \nRP Specials 147 \nTerm Financing versus Overnight Financing 150 \nShort Squeezes 150 \nThe Short Squeeze of 1986 \n151 \nTaking a Basis Trade into the Delivery Month 153 \nChapter 7 \nVolatility Arbitrage in the Treasury Bond Basis \n157 \nOverview \n158 \nThe Options Embedded in Bond Futures \n158 \nCalls, Puts, and Straddles \n159 \nTwo Arenas for Trading Volatility \n161 \nThe Option-Adjusted Bond Ba®is \n162 \nHistory of Mispricings \n164 \nVolatility Arbitrage \n165 \nReport Card \n167 \nExamples of Yield Enhancement \n169 \nLeverage \n170 \n. Words of Caution \n170 \nOther Applications \n171 \nChapter 8 \nNine Eras of the Bond Basis \n173 \nThe Birth and Maturation of Bond Futures \n173 \nV

In [6]:
%autoreload 2
#from pymongo import MongoClient
from raptor.raptor.chroma_manager import DatabaseConnectionManager
from raptor.raptor.tree_structures import Node

# Create a config with your custom models
db_path = "chromadb"
db_collection_name = 'samplefin'
mongodb_manager = DatabaseConnectionManager(db_config=(db_path, db_collection_name))

#mynode = Node(text="test_text", index=5001, children=[None], embeddings=None, db_manager=mongodb_manager)

2024-04-02 16:50:37,220 - Loading faiss with AVX2 support.
2024-04-02 16:50:37,238 - Successfully loaded faiss with AVX2 support.
2024-04-02 16:50:37,959 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [9]:
mongodb_manager.get_node_data(node_id= '01d6631d-763e-441e-8368-0ffd2d6a2776')

{'ids': ['01d6631d-763e-441e-8368-0ffd2d6a2776'],
 'embeddings': None,
 'metadatas': [{'children': '[]',
   'index': 831,
   'source': 'd50be7d7-fe3f-5342-95c6-147bb6170780'}],
 'documents': ['which was the cheapest to deliver on April 5 and one of which was  the on-the-run bond  Also provided are two futures DV01s, one  using the cheapest to deliver as the reference bond and one using  the on-the-run as the reference bond  Notice that the futures DVOl  with respect to a change in the on-the-run yield (132 92) equals the  product of the futures DV01 with respect to a change in the cheapest '],
 'uris': None,
 'data': None}

In [14]:
mongodb_manager.client.delete_collection("finbooks")

In [7]:
from raptor.raptor import GPT4QAModel, RetrievalAugmentationConfig, RetrievalAugmentation

mongodb_manager.tree_name = "sampilicious"
custom_config = RetrievalAugmentationConfig(
    mongodb_manager=mongodb_manager,
    qa_model=GPT4QAModel()
)

RA = RetrievalAugmentation(config=custom_config)

# construct the tree
RA.add_documents(text[6999:10000])
SAVE_PATH = "embeddings/finsample"
RA.save(SAVE_PATH)

2024-04-02 16:52:15,336 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptor.raptor.SummarizationModels.GPT3TurboSummarizationModel object at 0x7f11d3f0e4d0>
            Embedding Models: {'OpenAI': <raptor.raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x7f11d3f0fd60>}
            Cluster Embedding Model: OpenAI
        
        Reduction Dimension: 10
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2024-04-02 16:52:15,337 - Successfully initialized ClusterTreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K:

Mongodb connection test: 1712055139525497156


In [17]:
mongodb_manager.nodes_collection.peek()

{'ids': ['003183c0-4836-58f7-9fc5-6c7a6e388f85',
  '31231bfd-37ae-525c-bede-b43722d1fb19',
  '4f62d399-b2f6-562d-8ee4-b9c0c39c5494',
  '6610d432-8b32-532a-ac88-0a392bde1f50',
  '8a764bc5-4930-579d-9014-c044e3ada73f',
  'a778b66e-8df3-5c1d-ac6e-69ac5b11a6f6',
  'd91b849e-8d11-599f-a41b-5ef397539b7e',
  'e397e432-9c5d-5e70-9f97-cfeaa5408bcc',
  'fa92ca72-ab4b-5ea6-8811-6531d154fbbe'],
 'embeddings': [[-0.02826528064906597,
   -0.02522527612745762,
   0.006019207648932934,
   -0.016780821606516838,
   -0.01128854788839817,
   0.007640543393790722,
   -0.010383302345871925,
   -0.018983134999871254,
   -0.045262280851602554,
   -0.030859418213367462,
   0.010410324670374393,
   0.03861480578780174,
   -0.0035804491490125656,
   0.014011040329933167,
   0.02134758234024048,
   0.005002495367079973,
   0.006414408329874277,
   -0.022293362766504288,
   0.006397519260644913,
   0.008039121516048908,
   -0.031643062829971313,
   0.011950593441724777,
   -0.028292303904891014,
   0.002141513861

In [19]:
query_embeddings = mongodb_manager.nodes_collection.peek()['embeddings'][1]
mongodb_manager.nodes_collection.query(
        query_embeddings=[query_embeddings],
        n_results=5
        )

{'ids': [['31231bfd-37ae-525c-bede-b43722d1fb19',
   '8a764bc5-4930-579d-9014-c044e3ada73f',
   'fa92ca72-ab4b-5ea6-8811-6531d154fbbe',
   'a778b66e-8df3-5c1d-ac6e-69ac5b11a6f6',
   '4f62d399-b2f6-562d-8ee4-b9c0c39c5494']],
 'distances': [[0.0,
   0.26752450864405314,
   0.27897609706494747,
   0.28799482340908417,
   0.2936877472619686]],
 'metadatas': [[{'children': '[]', 'index': 7, 'source': 'sampilicious'},
   {'children': '[]', 'index': 2, 'source': 'sampilicious'},
   {'children': '[]', 'index': 1, 'source': 'sampilicious'},
   {'children': '[]', 'index': 8, 'source': 'sampilicious'},
   {'children': '[]', 'index': 5, 'source': 'sampilicious'}]],
 'embeddings': None,
 'documents': [["Bonds Exiting the Basket Trades 214  New Issuance 215  A Word of Caution  216  10  Applications for Portfolio Managers  217  Hedging and Asset Allocation  217  ,  Advantages of Using Futures for Hedging and Asset Allocation 218  Managing a Portfolio's Duration with Futures 219  Calculating the Durat

In [8]:
mongodb_manager.nodes_collection.count()

9

In [20]:
# finsample
ids = ['003183c0-4836-58f7-9fc5-6c7a6e388f85',
  '31231bfd-37ae-525c-bede-b43722d1fb19',
  '4f62d399-b2f6-562d-8ee4-b9c0c39c5494',
  '6610d432-8b32-532a-ac88-0a392bde1f50',
  '8a764bc5-4930-579d-9014-c044e3ada73f',
  'a778b66e-8df3-5c1d-ac6e-69ac5b11a6f6',
  'd91b849e-8d11-599f-a41b-5ef397539b7e',
  'e397e432-9c5d-5e70-9f97-cfeaa5408bcc',
  'fa92ca72-ab4b-5ea6-8811-6531d154fbbe']

for id in ids:
  node_data = mongodb_manager.get_node_data(id)
  if node_data['embeddings'] == None:
    print(id, "No embeddings")
  else:
    print(id, node_data['documents'])

003183c0-4836-58f7-9fc5-6c7a6e388f85 None
31231bfd-37ae-525c-bede-b43722d1fb19 None
4f62d399-b2f6-562d-8ee4-b9c0c39c5494 None
6610d432-8b32-532a-ac88-0a392bde1f50 None
8a764bc5-4930-579d-9014-c044e3ada73f None
a778b66e-8df3-5c1d-ac6e-69ac5b11a6f6 None
d91b849e-8d11-599f-a41b-5ef397539b7e None
e397e432-9c5d-5e70-9f97-cfeaa5408bcc None
fa92ca72-ab4b-5ea6-8811-6531d154fbbe None


In [48]:
# How to only get the nodes until text_size>n
qres = mongodb_manager.nodes_collection.get(ids=['5949fb67-ff6c-4484-b263-29ae59a51d59','60577be1-bc9a-4079-967e-36c99a163763',])

In [51]:
indexes = [data['index'] for data in qres['metadatas']]
qres

{'ids': ['5949fb67-ff6c-4484-b263-29ae59a51d59',
  '60577be1-bc9a-4079-967e-36c99a163763'],
 'embeddings': None,
 'metadatas': [{'children': '[]', 'index': 0}, {'children': '[]', 'index': 5}],
 'documents': ['17) and Leveraged Trading: A professional approach to trading FX, stocks on margin, CFDs, spread bets (Harriman House, 2019) Robert trades his own portfolio using the methods you can find in his books \x0cIntroduction I traded my first futures contract on instinct It was September 2002, and I had just stepped onto a trading floor for the first time  The floor was in Canary Wharf, one of London’s two key',
  '\x0cAfter leaving Barcap I spent a couple of years working in economic research before joining a hedge fund, AHL AHL was, and is, one of the largest futures traders in the world  My job was to develop and manage systematic futures trading strategies  Beginning with a relatively small portfolio, I was eventually promoted to manage the fixed income team, responsible for several 

In [7]:
from raptor.raptor import GPT4QAModel, RetrievalAugmentationConfig, RetrievalAugmentation

custom_config = RetrievalAugmentationConfig(
    mongodb_manager=mongodb_manager,
    qa_model=GPT4QAModel()
)


SAVE_PATH = "embeddings/chroma_sample"

RA = RetrievalAugmentation(config=custom_config, tree=SAVE_PATH)

2024-03-28 14:06:44,645 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptor.raptor.SummarizationModels.GPT3TurboSummarizationModel object at 0x7f14256cd060>
            Embedding Models: {'OpenAI': <raptor.raptor.EmbeddingModels.OpenAIEmbeddingModel object at 0x7f14256ced40>}
            Cluster Embedding Model: OpenAI
        
        Reduction Dimension: 10
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2024-03-28 14:06:44,647 - Successfully initialized ClusterTreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K:

Mongodb connection test: 1711613204652720341


In [9]:
q = "Describe the context"
RA.retrieve(question=q, collapse_tree=True)

2024-03-28 14:39:55,020 - Using collapsed_tree
2024-03-28 14:39:55,632 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 14:39:55,646 - Number of requested results 10 is greater than number of elements in index 7, updating n_results = 7


Pre-retrieval database check: 1711615195642012382
None


TypeError: 'NoneType' object is not subscriptable

# EmbedChain retrieve possible?

In [1]:
%load_ext autoreload

In [2]:
%autoreload
import pandas as pd
import os
from dotenv import load_dotenv
from embedchain import App
from embedchain.config import BaseLlmConfig
load_dotenv()

True

In [5]:
import yaml

def add_to_yaml_file(path, key, value):
    """
    Adds a key-value pair to a YAML file.

    Args:
    path (str): The path to the YAML file.
    key (str): The key of the key-value pair to add.
    value (any): The value associated with the key.
    """
    try:
        # Load the existing data from the file
        with open(path, 'r') as file:
            data = yaml.load(file, Loader=yaml.FullLoader) or {}
    except FileNotFoundError:
        data = {}
    
    # Add the new key-value pair
    data[key] = value

    # Write the updated dictionary back to the file
    with open(path, 'w') as file:
        yaml.dump(data, file)

def extract_from_yaml_file(path):
    """
    Extracts all key-value pairs from a YAML file as strings.

    Args:
    path (str): The path to the YAML file.

    Returns:
    dict: A dictionary of key-value pairs, with all values converted to strings.
    """
    try:
        # Read data from the file
        with open(path, 'r') as file:
            data = yaml.load(file, Loader=yaml.FullLoader)
        # Convert all values to strings
        return {k: str(v) for k, v in data.items()}
    except FileNotFoundError:
        return {}

# Example usage:
# add_to_yaml_file('example.yaml', 'new_key', 'This is a new value')
# extracted_data = extract_from_yaml_file('example.yaml')
# print(extracted_data)


In [3]:


def embedchain_bot(db_path, api_key):
    return App.from_config(
        config={
            "llm": {
                "provider": "openai",
                "config": {
                    "model": "gpt-3.5-turbo-1106",
                    "temperature": 0.5,
                    "max_tokens": 1000,
                    "top_p": 1,
                    "stream": True,
                    "api_key": api_key,
                },
            },
            "app": {
                "config": {
                    "log_level" : "DEBUG",
                    "id" : "finrag-100"
                }
            },
            "vectordb": {
                "provider": "chroma",
                "config": {"collection_name": "finbooks", "dir": db_path, "allow_reset": True},
            },
            "embedder": {"provider": "openai", "config": {"api_key": api_key}},
            "chunker": {"chunk_size": 1000, "chunk_overlap": 0, "length_function": "len"},
        }
    )

DB_PATH = "chromadb"
app = embedchain_bot(DB_PATH, os.getenv("OPENAI_API_KEY"))

In [3]:
# GPT-4
def embedchain_bot(db_path, api_key):
    return App.from_config(
        config={
            "llm": {
                "provider": "openai",
                "config": {
                    "model": "gpt-4-turbo-preview",
                    "temperature": 0.5,
                    "max_tokens": 1000,
                    "top_p": 1,
                    "stream": False,
                    "api_key": api_key,
                },
            },
            "app": {
                "config": {
                    "log_level" : "DEBUG",
                    "id" : "finrag-100"
                }
            },
            "vectordb": {
                "provider": "chroma",
                "config": {"collection_name": "finbooks", "dir": db_path, "allow_reset": True},
            },
            "embedder": {"provider": "openai", "config": {"api_key": api_key}},
            "chunker": {"chunk_size": 1000, "chunk_overlap": 0, "length_function": "len"},
        }
    )

DB_PATH = "chromadb"
app = embedchain_bot(DB_PATH, os.getenv("OPENAI_API_KEY"))

In [27]:
summary_source = '247a42e9-1bca-5ee6-b3df-9418d32794d3'
config = BaseLlmConfig(number_documents=20, stream=False)
where_clause = {
    "$and": [
        {
            "index": {"$gt": 2600},
        },
        {
            "children": {"$ne": '[]'},
        },
        {
            "source": {"$eq": summary_source}
        }
    ]
}
summary = app.chat("Write a summary of the entire contents of the book (based on the provided chunks)", citations=True, config=config, where=where_clause)

In [28]:
summary

('The book "Advanced Futures Trading Strategies" delves into a comprehensive analysis of various trading strategies tailored for futures trading, drawing from over 20 years of the author\'s experience in the field. It focuses on deliberate trading strategies backed by detailed analysis and historical data spanning over 50 years. The book covers essential topics such as contract selection, risk management, trade execution, and cash utilization, targeting experienced traders with a solid understanding of financial markets. It emphasizes the importance of diversification in a trader\'s toolbox and provides detailed explanations of different trading strategies, their historical performance, and key metrics like mean annual return, costs, drawdowns, Sharpe ratio, skew, and alpha. The author explores the application of trend filters, momentum trading strategies, and the significance of forecasting returns to avoid excessive portfolio exposure. Additionally, the book discusses the performance

In [26]:
add_to_yaml_file('./references/trees/top_level_summaries.yaml', summary_source, summary[0])

In [13]:
extracted_data = extract_from_yaml_file('./references/trees/example.yaml')
print(extracted_data.values())

dict_values(['The book delves into a wide array of topics related to financial markets, bond trading, asset allocation, and risk management. It covers detailed discussions on accrued interest, its impact on returns, and asset allocation strategies utilizing futures. Specific information about the Chicago Board of Trade (CBOT) is provided, including bond delivery requirements and conversion factors. The text also explores complex numerical analysis related to the futures market, with calculations involving factors like Futures DV01 and CTD DV. Additionally, the book focuses on hedging strategies in bond futures trading, emphasizing reckoning profit and loss of a hedge, evaluating hedge performance, working with durations, and using yield betas for effective hedging. The content highlights finding hedge ratios to align futures positions with bond yields and adjusting ratios based on yield betas for optimal hedging outcomes.', 'The book delves into a wide array of topics related to financ

In [5]:
config = BaseLlmConfig(number_documents=10, stream=False)
where_clause = {
    "index": {"$ne": 0}
}
app.chat("How can I design volatility arbitrage strategies for bond futures?", citations=True, config=config, where=where_clause)

('One approach to designing volatility arbitrage strategies for bond futures involves spreading cheap volatility in the basis market against rich volatility in the bond options market. This strategy entails buying the options embedded in bond futures by purchasing the bond basis and selling real or explicit options against this position. If executed correctly, such a volatility spread trade can profit from a narrowing in the implied volatility spread. This method is explained in detail in Chapter 7 of the text.',
 [('between the price paid for volatility in the bond basis market and  the price paid for volatility in the market for options on bond  futures  We were among those who knew how to do volatility arbi\xad trage, and we wrote about it as early as January 1990  During 1990  and 1991, as explained in Chapter 7, we put together a successful  track record that involved five or six trades a year  (See Exhibits 7 7 ',
   {'children': '[]',
    'index': 1266,
    'source': 'd50be7d7-f

In [6]:
config = BaseLlmConfig(number_documents=10, stream=False)
where_clause = {
    "index": {"$ne": 0}
}
app.chat("What is convexity bias in Eurodollar futures and how can I exploit it for profitable trading strategies?", citations=True, config=config, where=where_clause)

('Convexity bias in Eurodollar futures refers to the systematic advantage of being short Eurodollar futures compared to deposits, swaps, or FRAs. This bias is tied to interest rate volatility and time to futures expiration, affecting the value of the swap/Eurodollar rate spread. By understanding this bias and its implications, traders can potentially exploit it for profitable trading strategies, especially when hedging bonds with positive convexity. Additionally, trading the value of the swap/Eurodollar rate spread against options on forward Eurodollar rates can be a way to capitalize on this bias in the market.',
 [('The chapters discussed touch on various topics related to trading strategies and biases in the financial markets, particularly focusing on opportunities within the S&P Calendar Roll, trading the turn, and the convexity bias in Eurodollar futures. In "The Convexity Bias in Eurodollar Futures" (Chapter 7), authors Galen Burghardt and William Hoskins delve into the systemati

In [7]:
config = BaseLlmConfig(number_documents=10, stream=False)
where_clause = {
    "index": {"$ne": 0}
}
app.chat("How can I use PCA to create profitable trading strategies for bonds market?", citations=True, config=config, where=where_clause)

('You can utilize PCA in the bond market to identify trading opportunities by analyzing factors that influence trading outcomes, such as the net curvature in the PCA-neutral butterfly position. By conducting PCA on bond yield data, you can uncover underlying patterns and relationships that can guide your trading decisions. This approach can help you screen the market for trade ideas, select assets, and construct new types of trades based on the insights gained from PCA analysis. Additionally, PCA can assist in managing interest rate risk by hedging against changes in yield curve factors, allowing you to create non-directional positions that generate alpha and are immune to specific factors. By leveraging PCA in the bond market, you can potentially discover relative value trades and exploit mismatches between different issuers while being hedged against overall market moves.',
 [('The text discusses the application of Principal Component Analysis (PCA) on bond yields data to identify tr

In [23]:
config = BaseLlmConfig(number_documents=10, stream=False)
where_clause = {
    "index": {"$ne": 0}
}
app.chat("How can I design trend-following strategies for adjusted futures price data", citations=True, config=config, where=where_clause)

('Designing trend-following strategies for adjusted futures price data involves utilizing moving averages to identify trends in the market. By comparing the current price to a moving average of the back-adjusted futures price over a specific period, such as the last 12 months, you can determine if the market is in an uptrend. This approach allows you to capture the underlying trends in the instrument with positive carry, providing a more accurate picture of where your returns are coming from. Additionally, adjusting forecasts according to the level of volatility in each instrument can further enhance the effectiveness of trend-following strategies.',
 [('All other elements are identical to strategy four \x0c93 We use the back-adjusted futures price as it will be smooth, without disjointed sections when we have a futures roll  This is not so important for a 12-month moving average, but will be important once we’re considering shorter trends  A consequence of this is that we’ll be more l

In [32]:
import yaml
import numpy as np
from openai import OpenAI
import os

def embed(text):
    """
    Embeds the given text into a vector space. This is a placeholder function.
    You should replace this with your actual embedding function.
    """
    client = OpenAI()

    response = client.embeddings.create(
        input="Your text string goes here",
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

def cosine_similarity(vec1, vec2):
    """
    Computes the cosine similarity between two vectors.
    """
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def search_similarities(search_string, yaml_path):
    """
    Performs a cosine similarity search between a search string and each value in a YAML dictionary.

    Args:
    search_string (str): The string to search for.
    yaml_path (str): The path to the YAML file.

    Returns:
    dict: A dictionary with keys as YAML keys and values as similarity scores.
    """
    # Load data from the YAML file
    with open(yaml_path, 'r') as file:
        data = yaml.load(file, Loader=yaml.FullLoader) or {}

    # Embed the search string
    search_vector = embed(search_string)
    similarities = {}

    # Compute similarity with each entry in the YAML file
    for key, value in data.items():
        value_vector = embed(str(value))  # Ensure the value is a string
        similarity = cosine_similarity(search_vector, value_vector)
        similarities[key] = similarity

    return similarities


In [33]:
# Example usage:
results = search_similarities('Suggest a systematic trading strategy to use in futures market', './references/trees/top_level_summaries.yaml')
print(results)

{'247a42e9-1bca-5ee6-b3df-9418d32794d3': 0.9999999999999999, 'b08eb3fd-421a-5eeb-82c1-a683d13a2bde': 0.9999999999999999, 'd50be7d7-fe3f-5342-95c6-147bb6170780': 0.9999999999999999, 'd6d86700-20f3-57aa-87c7-21fc81e34fea': 0.9999999999999999}


In [11]:
## Other commands that can be accessed from embedchain app:
# app.db.query(input_query="Which hedge funds did the author work at?", n_results=2, where={'app_id':app.config.id}, citations=False)
# app.search("What are the names of the hedge funds where the author worked?", num_documents=2)

In [24]:
from embedchain.helpers.callbacks import (StreamingStdOutCallbackHandlerYield,
                                          generate)

def query_config():
    DEFAULT_PROMPT_WITH_HISTORY = """
    You are a financial Q&A expert system. Your responses must always be rooted in the context provided for each query.
    You are also provided with the conversation history with the user. Make sure to use relevant context from conversation history as needed.

    Here are some guidelines to follow:

    1. Refrain from explicitly mentioning the context provided in your response.
    2. The context should silently guide your answers without being directly acknowledged.
    3. Do not use phrases such as 'According to the context provided', 'Based on the context, ...' etc.

    Context information:
    ----------------------
    $context
    ----------------------

    Conversation history:
    ----------------------
    $history
    ----------------------

    Query: $query
    Answer:
    """ 
    config = BaseLlmConfig(number_documents=3,
                           prompt=DEFAULT_PROMPT_WITH_HISTORY)
    return config

app.query("What are the names of the hedge funds where the author worked?", citations=True, config=query_config(), where=where_clause)
#app.query("What are the names of the hedge funds where the author worked?", citations=True, where=where_clause)

TypeError: can only join an iterable